# PDF 문서를 내가 원하는 방식으로 요약하기

In [1]:
%pip install PyMuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.6/18.6 MB 9.1 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## PyMuPDF로 PDF 파일 열기
- 페이지별로 텍스트를 불러올 수 있다. 
- 하지만 헤더와 푸터의 내용까지 포함되어 있다. 

In [4]:
import pymupdf

pdf_file_path = './data/인공지능 기법을 활용한 농촌지역의 객체 정보 추출방안.pdf'
doc = pymupdf.open(pdf_file_path)

for page in doc:
    print(page.get_text())
    print('-------------------' * 2)

10
▶
인공지능 기법을 활용한 농촌지역의 
객체 정보 추출방안
1. 머리말
최근 4차 산업혁명 기술이 경제 화두로 부상한 이후 3D, 빅데이터, IoT, 
AI를 활용한 사업들이 성장세를 띄고 있으며, 특히 AI 기술은 스마트도시, 
스마트건설, 보건복지, 안전 등 다양한 분야로 활용되고 있다. 이러한 4차 
산업혁명 기술은 농업 및 농촌분야에서도 빠르게 적용되고 있다. 농촌분야
에서는 ICT, 공간빅데이터 등을 활용해 증거기반 주민체감형 지역개발과 
같은 지능형 스마트 농촌을 구현하기 위한 연구가 수행되고 있으며, 농업
분야에서는 빅데이터, AI, 사물인터넷 등 ICT 기술을 접목한 스마트팜 연
구가 수행되고 있다. 이처럼 농업과 농촌 분야에 신기술의 도입으로 학문
분야의 확장과 새로운 비즈니스를 창출하고 있다. 
이러한 변화속에서 정부는 농촌공간의 쾌적성과 편리성을 높이고, 농촌
주민 삶의 질 향상을 위해 농산촌 지원강화 및 성장환경 조성이라는 국정
목표를 세우고, 이에 대한 세부실천 목표를 제시하였다. 이 가운데 하나가 
농촌공간 재구조화 계획이다. 이 계획은 농촌공간의 여건에 따라 주거·
생산·서비스 등 기능적으로 구분되고 재배치될 수 있도록 장기계획 수립 
및 농촌특화지구 도입을 목표로 하고 있다. 이를 위해 「농촌공간 재구조화 
및 재생지원을 위한 법률」(농촌공간재구조화법)을 22년도에 제정하였으며, 
24년부터 시행에 들어가게 된다. 또한 장기적인 계획에 대응하여 서비스 
거점 확충, 주거지 인접 공장·축사 정비·집적화 등을 지원하는 농촌재생 
프로젝트를 추진하고자 하고 있다. 
농촌지역을 대상으로 국정목표에 따른 세부실천과제를 수행하기 위해서
는 기존의 현황에 대한 파악이 우선되어야 한다. 현재 우리나라의 농촌지
역에 해당하는 공간은 국토기본법, 국토의 계획 및 이용에 관한 법률(국토
02
전 정 배
공간정보연구원 
/ 선임연구원
jbjeon@lx.or.kr
석 승 원
전북대학교 스마트팜학과 
/ 학사과정
champ9162@jbnu.ac.kr


## 텍스트 영역의 글자만 불러오기

In [6]:
import os

def extract_text_from_pdf(
        pdf_file_path, 
        header_height,  # 헤더의 높이 
        footer_height,  # 푸터의 높이
    ):
        
    doc = pymupdf.open(pdf_file_path)

    full_text = ''

    for page in doc:
        rect = page.rect # 페이지의 크기를 가져온다.
        
        header = page.get_text(clip=(0, 0, rect.width , header_height))
        footer = page.get_text(clip=(0, rect.height - footer_height, rect.width , rect.height))
        text = page.get_text(clip=(0, header_height, rect.width , rect.height - footer_height))

        full_text += text + '\n------------------------------------\n'

    # 파일명만 추출
    pdf_file_name = os.path.basename(pdf_file_path)
    txt_file_path = f'data/output/{pdf_file_name}_with_preprocessing.txt'

    with open(txt_file_path, 'w', encoding='utf-8') as f:
        f.write(full_text)

    return txt_file_path


txt_file_path = extract_text_from_pdf(
    pdf_file_path, 
    header_height=80,  # 헤더의 높이 
    footer_height=40,  # 푸터의 높이
)

print(f'Extracted text saved to: {txt_file_path}')

Extracted text saved to: data/output/인공지능 기법을 활용한 농촌지역의 객체 정보 추출방안.pdf_with_preprocessing.txt


In [10]:
from openai import OpenAI

def summarize_txt(
        file_path: str,
        model: str = "gpt-4o-mini", # OpenAI model
    ):
    client = OpenAI()

    # (2) 주어진 텍스트 파일을 읽어들인다.
    with open(file_path, 'r', encoding='utf-8') as f:
        txt = f.read()

    # (3) 요약을 위한 시스템 프롬프트를 생성한다.
    system_prompt = f'''
    너는 다음 글을 요약하는 봇이다. 아래 글을 읽고, 저자의 문제 인식과 주장을 파악하고, 주요 내용을 요약하라. 

    작성해야 하는 포맷은 다음과 같다. 
    
    # 제목

    ## 요약 (목적, 방법, 결과, 의의를 리스트로 정리한다.)

    ## 주요 키워드 (문서를 대표하는 키워드를 5개 이하로 추출한다.)
    
    ## 저자 소개 (이름, 소속, 직급, 연락처 정보를 리스트로 정리)

    
    =============== 이하 텍스트 ===============

    { txt }
    '''

    # (4) OpenAI API를 사용하여 요약을 생성한다.
    response = client.chat.completions.create(
        model=model,
        temperature=0.1,
        messages=[
            {"role": "system", "content": system_prompt},
        ]
    )

    return response.choices[0].message.content

In [11]:
def summarize_document(
        pdf_file_path: str,
        header_height: int,
        footer_height: int
    ):    

    txt_file_path = extract_text_from_pdf(
        pdf_file_path, header_height, footer_height
    )

    print(f"Text extracted from {pdf_file_path} is saved")

    summary = summarize_txt(txt_file_path)
    print(summary)

    # (5) 요약된 내용을 파일로 저장한다.
    summary_file_path = txt_file_path.replace('.txt', f'_summary.txt')
    with open(summary_file_path, 'w', encoding='utf-8') as f:
        f.write(summary)

    print(f"Summary is saved to {summary_file_path}")

    return summary_file_path

In [12]:
summarize_document(
    pdf_file_path=pdf_file_path,
    header_height=80,  # 헤더의 높이 
    footer_height=40,  # 푸터의 높이
)

Text extracted from ./data/인공지능 기법을 활용한 농촌지역의 객체 정보 추출방안.pdf is saved
# 인공지능 기법을 활용한 농촌지역의 객체 정보 추출방안

## 요약
- **목적**: 농촌공간의 현황을 정확히 파악하고, 이를 기반으로 농촌공간 재구조화 계획을 수립하기 위한 방법론 제안.
- **방법**: YOLO 알고리즘을 활용하여 드론으로 촬영된 영상을 분석하고, 건축물과 비닐하우스의 객체 탐지를 수행.
- **결과**: 1,147개의 건축물과 297개의 비닐하우스를 탐지하였으며, 기존 데이터와 비교하여 96.8%의 정확도로 객체를 인식함. 그러나 일부 비닐하우스와 불법 건축물에 대한 추가 검토가 필요함.
- **의의**: 인공지능 기법을 통해 농촌공간의 실제 현황을 객관적으로 파악할 수 있으며, 법적 규제사항 검토와 농촌공간 재구조화에 기여할 수 있는 기초 자료를 제공.

## 주요 키워드
- 인공지능
- YOLO 알고리즘
- 농촌공간 재구조화
- 객체 탐지
- 드론 영상

## 저자 소개
- **전정배**
  - 소속: 공간정보연구원
  - 직급: 선임연구원
  - 연락처: jbjeon@lx.or.kr

- **석승원**
  - 소속: 전북대학교 스마트팜학과
  - 직급: 학사과정
  - 연락처: champ9162@jbnu.ac.kr

- **김솔희**
  - 소속: 전북대학교 스마트팜학과
  - 직급: 연구교수
  - 연락처: solhee1101@jbnu.ac.kr

- **김태곤**
  - 소속: 전북대학교 스마트팜학과
  - 직급: 교수
  - 연락처: taegon@jbnu.ac.kr
Summary is saved to data/output/인공지능 기법을 활용한 농촌지역의 객체 정보 추출방안.pdf_with_preprocessing_summary.txt


'data/output/인공지능 기법을 활용한 농촌지역의 객체 정보 추출방안.pdf_with_preprocessing_summary.txt'

In [14]:
from glob import glob

for g in glob('./data/*.pdf'):
    print(g)
    summarize_document(
        pdf_file_path=g,
        header_height=80,  # 헤더의 높이 
        footer_height=40,  # 푸터의 높이
    )
    print('-------------------' * 2)

./data/생성형 AI 기반의 영농 의사결정 지원 시스템 개발과 향후계획.pdf
Text extracted from ./data/생성형 AI 기반의 영농 의사결정 지원 시스템 개발과 향후계획.pdf is saved
# 생성형 AI 기반의 영농 의사결정 지원 시스템 개발과 향후계획

## 요약
- **목적**: 생성형 AI와 영농 의사결정 지원 시스템의 연계를 통해 농업인의 영농 의사결정을 지원하고, 기후변화에 적응할 수 있는 시스템 개발.
- **방법**: APEX 모델을 활용한 기후변화 평가 후, 농업인에게 유용한 영농 의사결정 지원 프로그램 '팜두레' 개발. ChatGPT와 API 연계를 통해 지역 맞춤형 정보 제공.
- **결과**: AI 팜두레는 작물 재배 일정 및 기후 정보를 바탕으로 영농 일정을 작성하고, 사용자와의 쌍방향 소통을 통해 영농 계획을 수정할 수 있는 기능을 갖춤.
- **의의**: 생성형 AI를 활용한 영농 의사결정 지원 시스템은 농업 생산성을 높이고, 기후변화에 효과적으로 대응할 수 있는 가능성을 제시.

## 주요 키워드
- 생성형 AI
- 영농 의사결정
- 팜두레
- 기후변화
- 스마트농업

## 저자 소개
- **최순군**
  - 소속: 국립농업과학원
  - 직급: 농업연구사
  - 연락처: soonkun@korea.kr

- **이병모**
  - 소속: 국립농업과학원
  - 직급: 농업연구관
  - 연락처: leebm@korea.kr

- **전상민**
  - 소속: 국립농업과학원
  - 직급: 농업연구사
  - 연락처: luckyjsm@korea.kr

- **이준혁**
  - 소속: ㈜노트스퀘어
  - 직급: 대표
  - 연락처: jlee@notesquare.co.kr
Summary is saved to data/output/생성형 AI 기반의 영농 의사결저